# Объединения и слияния наборов данных

Здесь мы рассмотрим несколько методов по объединению и слиянию нескольких наборов данных, а также рассмотрим некоторые неочевидные проблемы, которые могут возникать. 

In [1]:
import numpy as np
import pandas as pd

In [2]:
# создадим простую функцию для быстрого создания датафрема
def make_df(cols, ind):
    data = {c: [str(c) + str(i) for i in ind]
            for c in cols}
    return pd.DataFrame(data, ind)

make_df('ABC', range(3))

,A,B,C
0,A0,B0,C0
1,A1,B1,C1
2,A2,B2,C2


# 1. Метод ``pd.concat`` для объединения

In [4]:
ser1 = pd.Series(['A', 'B', 'C'], index=[1, 2, 3])
ser2 = pd.Series(['D', 'E', 'F'], index=[4, 5, 6])
pd.concat([ser1, ser2])

1    A
2    B
3    C
4    D
5    E
6    F
dtype: object

In [5]:
df1 = make_df('AB', [1, 2])
df2 = make_df('AB', [3, 4])
display(df1, df2, pd.concat([df1, df2]))

,A,B
1,A1,B1
2,A2,B2


,A,B
3,A3,B3
4,A4,B4


,A,B
1,A1,B1
2,A2,B2
3,A3,B3
4,A4,B4


In [6]:
# меняем ось
df3 = make_df('AB', [0, 1])
df4 = make_df('CD', [0, 1])
display(df3, df4, pd.concat([df3, df4], axis=1))

,A,B
0,A0,B0
1,A1,B1


,C,D
0,C0,D0
1,C1,D1


,A,B,C,D
0,A0,B0,C0,D0
1,A1,B1,C1,D1


## Проблема 1: Дублирование индексов

Первая проблема, которую стоит отметить, это дублирование индексов. То есть если вдруг получилось так, что ваши _DataFrame_ имеют одинаковые индексы (а это почти всегда так), то при объединении по умолчанию индексы просто продублируются и делать в дальнейшем срезы будет неудобно.

In [7]:
x = make_df('AB', [0, 1])
y = make_df('AB', [2, 3])
y.index = x.index 
display(x, y, pd.concat([x, y]))

,A,B
0,A0,B0
1,A1,B1


,A,B
0,A2,B2
1,A3,B3


,A,B
0,A0,B0
1,A1,B1
0,A2,B2
1,A3,B3


Рассмотрим несколько способов решений

#### `verify_integrity`, чтобы при дублировании возникала ошибка

In [8]:
pd.concat([x, y], verify_integrity=True)

ValueError: Indexes have overlapping values: Int64Index([0, 1], dtype='int64')

#### `ignore_index` , чтобы индексы пересчитались

In [9]:
display(x, y, pd.concat([x, y], ignore_index=True))

,A,B
0,A0,B0
1,A1,B1


,A,B
0,A2,B2
1,A3,B3


,A,B
0,A0,B0
1,A1,B1
2,A2,B2
3,A3,B3


#### `keys` для создания мультииндексов

In [10]:
display(x, y, pd.concat([x, y], keys=['x', 'y']))

,A,B
0,A0,B0
1,A1,B1


,A,B
0,A2,B2
1,A3,B3


A   B
x 0  A0  B0
  1  A1  B1
y 0  A2  B2
  1  A3  B3

## Проблема 2: Разные столбцы

Вторая проблема - это разные столбцы у датафреймов. В таком случае могут появиться пропущенные значения, об обработке которых мы говорили на прошлом занятии

In [11]:
# выводим только пересечение
df5 = make_df('ABC',[1,2])
df6 = make_df('BCD',[3,4])
display(df5, df6,
        pd.concat([df5, df6]))

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:5: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """


,A,B,C
1,A1,B1,C1
2,A2,B2,C2


,B,C,D
3,B3,C3,D3
4,B4,C4,D4


,A,B,C,D
1,A1,B1,C1,NaN
2,A2,B2,C2,NaN
3,NaN,B3,C3,D3
4,NaN,B4,C4,D4


Можно вывести только пересечение столбцов. В таком случае не будут учтены столбцы с потенциально пропущенными значениями

In [12]:
# выводим только пересечение
df5 = make_df('ABC',[1,2])
df6 = make_df('BCD',[3,4])
display(df5, df6,
        pd.concat([df5, df6], join='inner'))

,A,B,C
1,A1,B1,C1
2,A2,B2,C2


,B,C,D
3,B3,C3,D3
4,B4,C4,D4


,B,C
1,B1,C1
2,B2,C2
3,B3,C3
4,B4,C4


In [13]:
# выводим только столбцы df5
display(df5, df6,
        pd.concat([df5, df6], join_axes=[df5.columns]))

,A,B,C
1,A1,B1,C1
2,A2,B2,C2


,B,C,D
3,B3,C3,D3
4,B4,C4,D4


,A,B,C
1,A1,B1,C1
2,A2,B2,C2
3,NaN,B3,C3
4,NaN,B4,C4


# 2. Метод ``append()`` для объединения
Еще один метод для соединения двух наборов данных. 

Несмотря на синтаксис `data_1.append(data_2)`, метод `append` не изменяет исходный DataFrame

In [14]:
display(df1, df2, df1.append(df2))

,A,B
1,A1,B1
2,A2,B2


,A,B
3,A3,B3
4,A4,B4


,A,B
1,A1,B1
2,A2,B2
3,A3,B3
4,A4,B4


# 3. Слияние и соединение с помощью `join` и `merge`

## 3.1 Виды соединений

**"один к одному"**

Предположим, у нас есть 2 набора данных: 1ый отражает имя студента и то, в какую группу на своем потоке он входит, 2ой - также имя и его средний учебный балл 

In [15]:
df1 = pd.DataFrame({'Student': ['Tom', 'Ujin', 'Ann', 'Polina','Sam'],
                    'group': ['01', '02', '02', '01','02']})
df2 = pd.DataFrame({'Student': ['Tom', 'Ujin', 'Ann', 'Polina','Sam'],
                    'GPA': ['7.8', '6.4', '8.3', '9','8.2']})
display(df1, df2)

,Student,group
0,Tom,01
1,Ujin,02
2,Ann,02
3,Polina,01
4,Sam,02


,Student,GPA
0,Tom,7.8
1,Ujin,6.4
2,Ann,8.3
3,Polina,9
4,Sam,8.2


Соединим 2 набора данных. Видим, что слияние автоматически произошло по одинаковому столбцу

In [16]:
df3 = pd.merge(df1, df2)
df3

,Student,group,GPA
0,Tom,01,7.8
1,Ujin,02,6.4
2,Ann,02,8.3
3,Polina,01,9
4,Sam,02,8.2


**"много к одному"**

Теперь предположим, что у нас есть еще один DataFrame, отражающий имя куратора в каждой из двух групп. При слиянии видим, что по общему столбцу **group** произошло соединение 2х наборов данных, причем данные из 2ого набора повторяются.

In [17]:
df4 = pd.DataFrame({'group': ['01', '02'],
                    'tutor': ['Ahmed', 'Vova']})
display(df3, df4, pd.merge(df3, df4))

,Student,group,GPA
0,Tom,01,7.8
1,Ujin,02,6.4
2,Ann,02,8.3
3,Polina,01,9
4,Sam,02,8.2


,group,tutor
0,01,Ahmed
1,02,Vova


,Student,group,GPA,tutor
0,Tom,01,7.8,Ahmed
1,Polina,01,9,Ahmed
2,Ujin,02,6.4,Vova
3,Ann,02,8.3,Vova
4,Sam,02,8.2,Vova


**"многие ко многим"**

Нам прислали еще данные. На этот раз мы имеем информацию о предметах, которые студенты посещают в каждой из групп. При слиянии видим, что имя каждого студента продублировалось, так как для каждой группы (колонки, объединяющей студентов с предметами) имеется по 2 предмета

In [18]:
df5 = pd.DataFrame({'group': ['01', '01',
                              '02', '02'],
                    'subjects': ['math', 'IT', 'english', 'microeconomics',]})
display(df1, df5, pd.merge(df1, df5))

,Student,group
0,Tom,01
1,Ujin,02
2,Ann,02
3,Polina,01
4,Sam,02


,group,subjects
0,01,math
1,01,IT
2,02,english
3,02,microeconomics


,Student,group,subjects
0,Tom,01,math
1,Tom,01,IT
2,Polina,01,math
3,Polina,01,IT
4,Ujin,02,english
5,Ujin,02,microeconomics
6,Ann,02,english
7,Ann,02,microeconomics
8,Sam,02,english
9,Sam,02,microeconomics


## 3.2 Ключ слияния

А что делать, если хочется самому выбрать название столбца, по которому будет происходить слияние. Подумайте, когда это может быть нужно

### Ключевое слово `on`

In [19]:
display(df1, df2, pd.merge(df1,df2, on ='Student'))

,Student,group
0,Tom,01
1,Ujin,02
2,Ann,02
3,Polina,01
4,Sam,02


,Student,GPA
0,Tom,7.8
1,Ujin,6.4
2,Ann,8.3
3,Polina,9
4,Sam,8.2


,Student,group,GPA
0,Tom,01,7.8
1,Ujin,02,6.4
2,Ann,02,8.3
3,Polina,01,9
4,Sam,02,8.2


### Ключевые слова `left_on`, `right_on`

Можно соединять и по разным по названию столбцам. Для этого нужно указать какой столбец берется из "левого" df, а какой из "правого"

Здесь новая табличка отображает стипендию каждого студента

In [20]:
df3 = pd.DataFrame({'name': ['Tom', 'Ujin', 'Ann', 'Polina','Sam'],
                    'salary': [1600, 0, 1600, 2000,10000,]})
display(df1, df3, pd.merge(df1, df3, left_on="Student", right_on="name").drop('name',axis=1))

,Student,group
0,Tom,01
1,Ujin,02
2,Ann,02
3,Polina,01
4,Sam,02


,name,salary
0,Tom,1600
1,Ujin,0
2,Ann,1600
3,Polina,2000
4,Sam,10000


,Student,group,salary
0,Tom,01,1600
1,Ujin,02,0
2,Ann,02,1600
3,Polina,01,2000
4,Sam,02,10000


### Ключевые слова `left_index`, `right_index`

Соединять можно не только по названию столбцов, но и по названию индексов.

In [21]:
# по названию индекса
df1a = df1.set_index('Student')
df2a = df2.set_index('Student')
display(df1a, df2a)

,group
Student,
Tom,01
Ujin,02
Ann,02
Polina,01
Sam,02


,GPA
Student,
Tom,7.8
Ujin,6.4
Ann,8.3
Polina,9
Sam,8.2


In [22]:
# по названию индекса
display(df1a, df2a,
        pd.merge(df1a, df2a, left_index=True, right_index=True))

,group
Student,
Tom,01
Ujin,02
Ann,02
Polina,01
Sam,02


,GPA
Student,
Tom,7.8
Ujin,6.4
Ann,8.3
Polina,9
Sam,8.2


,group,GPA
Student,,
Tom,01,7.8
Ujin,02,6.4
Ann,02,8.3
Polina,01,9
Sam,02,8.2


Метод `join` делает тоже самое по названию индекса (ничего указывать не надо)

In [23]:
display(df1a, df2a, df1a.join(df2a))

,group
Student,
Tom,01
Ujin,02
Ann,02
Polina,01
Sam,02


,GPA
Student,
Tom,7.8
Ujin,6.4
Ann,8.3
Polina,9
Sam,8.2


,group,GPA
Student,,
Tom,01,7.8
Ujin,02,6.4
Ann,02,8.3
Polina,01,9
Sam,02,8.2


### `left/right_index` + `left/right_on`

In [24]:
display(df1a, df3, pd.merge(df1a, df3, left_index=True, right_on='name'))

,group
Student,
Tom,01
Ujin,02
Ann,02
Polina,01
Sam,02


,name,salary
0,Tom,1600
1,Ujin,0
2,Ann,1600
3,Polina,2000
4,Sam,10000


,group,name,salary
0,01,Tom,1600
1,02,Ujin,0
2,02,Ann,1600
3,01,Polina,2000
4,02,Sam,10000


## 3.3 Задание операций над множествами 

Пусть у нас есть 2 таблички, в них есть одинаковый столбец, но в нем есть несовпадающие значения. В таком случае удобно получить либо только совпадающие значения, либо все, либо значения только из правого/левого _DataFrame_

In [25]:
# по умолчанию - пересечение
df6 = pd.DataFrame({'Student': ['Tom', 'Ann', 'Sam'],
                    'subjects': ['maths', 'micro', 'macro']},
                   columns=['Student', 'subjects'])
df7 = pd.DataFrame({'Student': ['Tom', 'Ujin'],
                    'salary': ['1600', '0']},
                   columns=['Student', 'salary'])
display(df6, df7, pd.merge(df6, df7))

,Student,subjects
0,Tom,maths
1,Ann,micro
2,Sam,macro


,Student,salary
0,Tom,1600
1,Ujin,0


,Student,subjects,salary
0,Tom,maths,1600


In [26]:
# исправим на объединение
display(df6, df7, pd.merge(df6, df7, how='outer'))

,Student,subjects
0,Tom,maths
1,Ann,micro
2,Sam,macro


,Student,salary
0,Tom,1600
1,Ujin,0


,Student,subjects,salary
0,Tom,maths,1600
1,Ann,micro,NaN
2,Sam,macro,NaN
3,Ujin,NaN,0


In [27]:
# left , right соответственно выбирает строки для левого и правого df
display(df6, df7, pd.merge(df6, df7, how='left'))

,Student,subjects
0,Tom,maths
1,Ann,micro
2,Sam,macro


,Student,salary
0,Tom,1600
1,Ujin,0


,Student,subjects,salary
0,Tom,maths,1600
1,Ann,micro,NaN
2,Sam,macro,NaN


## 3.4 Еще одна проблема - столбцы пересекаются

Предположим теперь, что у нас есть данные по студентам и по предметам, которые они посещают. Первая табличка - предметы, которые студенты посещают в понедельник, вторая - во вторник. Но вот проблема, data-monkey, который предоставил нам данные, забыл в названии столбцов указать день недели. В следствие этого, столбцы стали пересекаться по названию

In [28]:
df8 = pd.DataFrame({'student': ['Max', 'Ujin', 'Polina', 'Sam'],
                    'subjects': ['Math', 'It', 'English', 'Latex']})
df9 = pd.DataFrame({'student': ['Max', 'Ujin', 'Polin', 'Sam'],
                    'subjects': ['IT', 'Math', 'Micro', 'Macro']})
display(df8, df9, pd.merge(df8, df9, on="student"))

,student,subjects
0,Max,Math
1,Ujin,It
2,Polina,English
3,Sam,Latex


,student,subjects
0,Max,IT
1,Ujin,Math
2,Polin,Micro
3,Sam,Macro


,student,subjects_x,subjects_y
0,Max,Math,IT
1,Ujin,It,Math
2,Sam,Latex,Macro


Видим, что ничего страшного не произошло, метод просто добавил 2 столбца, правда не очень понятно, какой столбец соответствует какому датафрейму. Для того чтобы это исправить, можно воспользоваться параметром `suffixes`

In [29]:
display(df8, df9, pd.merge(df8, df9, on="student", suffixes=["_L", "_R"]))

,student,subjects
0,Max,Math
1,Ujin,It
2,Polina,English
3,Sam,Latex


,student,subjects
0,Max,IT
1,Ujin,Math
2,Polin,Micro
3,Sam,Macro


,student,subjects_L,subjects_R
0,Max,Math,IT
1,Ujin,It,Math
2,Sam,Latex,Macro
